# Hybrid recommender system
The system makes recommendations based on the movies metadata and collaborative filtering

## Import recommenders

In [3]:
%%capture
%run collaborative_filtering.ipynb
%run metadata_based_recommender.ipynb

## Import libraries

In [4]:
from operator import itemgetter

## Peek at the data

In [5]:
movies.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,tagline,title,video,vote_average,vote_count,cast,crew,keywords,director,soup
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,NaN,Toy Story,False,7.70000000000000017763568394002504646778106689...,5415.0,"[tomhanks, timallen, donrickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousi, toy, boy, friendship, friend, rival...","[johnlasseter, johnlasseter, johnlasseter]",Animation Comedy Family johnlasseter johnlasse...


In [6]:
ratings.head(1)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144


# Hybrid recommender

In [7]:
def predict_ratings(movies_,user_id):
    res = []
    user_data = ratings[ratings['userId']==user_id]
    for idx in range(len(movies_)):
        movie_id = movies[movies['title']==movies_.iloc[idx]['title']]['id'].iat[0]
        if movie_id in user_data['movieId']:
            curr_r_ui = user_data[user_data['movieId']==movie_id]['rating']
            res.append([idx,collaborative_filtering.predict(uid=user_id,iid=movie_id,r_ui=curr_r_ui).est])
        else:
            res.append([idx,collaborative_filtering.predict(uid=user_id,iid=movie_id).est])
    return sorted(res,key = itemgetter(1),reverse=True)

def hybrid_recommender(film_name, user_id):
    metadata_recommendation = metadata_based_recommender(film_name)
    predicted_ratings = predict_ratings(metadata_recommendation,user_id)
    return metadata_recommendation.iloc[np.array(predicted_ratings)[:,0]]

# Make some predictions

In [8]:
hybrid_recommender('Jumanji',1).head()

,title,vote_count,vote_average,genres
4771,Harry Potter and the Philosopher's Stone,7188.0,7.50000000000000000000000000000000000000000000...,"[Adventure, Fantasy, Family]"
13908,Harry Potter and the Half-Blood Prince,5435.0,7.40000000000000035527136788005009293556213378...,"[Adventure, Fantasy, Family]"
17462,Captain America: The First Avenger,7174.0,6.59999999999999964472863211994990706443786621...,"[Action, Adventure, Science Fiction]"
4514,Jurassic Park III,2109.0,5.70000000000000017763568394002504646778106689...,"[Adventure, Action, Thriller, Science Fiction]"
16145,Harry Potter and the Deathly Hallows: Part 1,5708.0,7.50000000000000000000000000000000000000000000...,"[Adventure, Fantasy, Family]"
